In [51]:
import time
import docker 
import re
import os
import struct
import queue

class DockerStreamWrapper:
    def __init__(self, exec_id, stream_type, sock):
        self.exec_id = exec_id
        self.stream_type = stream_type
        self._sock = sock
        self._buffer = ""
        self._line_queue = queue.Queue()

    def demux_docker_stream(self, data):
        stdout = ""
        stderr = ""
        offset = 0
        while offset + 8 <= len(data):
            header = data[offset:offset + 8]
            stream_type, length = struct.unpack('>BxxxL', header)
            offset += 8
            chunk = data[offset:offset + length].decode('utf-8')
            offset += length
            # Remove ANSI escape codes
            chunk = re.sub(r'\x1B[@-_][0-?]*[ -/]*[@-~]', '', chunk)
            if stream_type == 1:
                stdout += chunk
            elif stream_type == 2:
                stderr += chunk

        if stdout and not stdout.endswith('\n'):
            stdout += '\n'
        if stderr and not stderr.endswith('\n'):
            stderr += '\n'
        return stdout, stderr

    def readline(self):
        while self._line_queue.empty():
            raw_data = self._sock.recv(2048)
            print(raw_data.hex())
            stdout, stderr = self.demux_docker_stream(raw_data)
            new_data = stdout if self.stream_type == 'stdout' else stderr
            self._buffer += new_data

            while '\n' in self._buffer:
                newline_pos = self._buffer.find('\n')
                line = self._buffer[:newline_pos]
                self._buffer = self._buffer[newline_pos + 1:]
                self._line_queue.put(line.strip())

        return self._line_queue.get()

    def write(self, data):
        self._sock.sendall(data.encode())

    def flush(self):
        pass





class DockerProcWrapper:
    def __init__(self, command, session_path):
        self.client = docker.from_env()
        self.low_client = docker.APIClient()
        self.image_name = "openinterpreter-runtime-container:latest"
        self.session_path = session_path
        self.id = os.path.basename(session_path)

        # Initialize exec instance
        self.init_exec_instance(command)

        self._sock = self.low_client.exec_start(self.exec_id, socket=True)._sock

        self.stdin = DockerStreamWrapper(self.exec_id, 'stdin', self._sock)
        self.stdout = DockerStreamWrapper(self.exec_id, 'stdout', self._sock)
        self.stderr = DockerStreamWrapper(self.exec_id, 'stderr', self._sock)

    def init_exec_instance(self, command):
        # Search for an existing container with the same session ID
        existing_containers = self.client.containers.list(filters={"label": f"session_id={self.id}"}, all=True)

        if existing_containers:
            container = existing_containers[0]
            if container.status != 'running' :
                container.start()
                timeout_seconds = 10  # 10 seconds timeout
                start_time = time.time()

                while container.status != 'running':
                    elapsed_time = time.time() - start_time
                    if elapsed_time > timeout_seconds:
                        print("Error: Timed out waiting for container to start.")
                        return  # or raise an exception

                    print(f"Waiting for container to start... (Status: {container.status})")
                    time.sleep(1)
                    container.reload()
        else:
            container = self.client.containers.run(
                self.image_name,
                detach=True,
                command="/bin/bash -i",  # Keep the container running with bash
                labels={'session_id': self.id},
                volumes={self.session_path: {'bind': '/mnt/data', 'mode': 'rw'}},
                user="nobody",  # Run as non-root user for security,
                stdin_open = True,
                tty=True
            )
            # Wait for container to start running
            while container.status != 'running' and container.status != "exited":
                time.sleep(1)
                container.reload()

        self.exec_id = self.low_client.exec_create(
            container.id,
            cmd=command,
            stdin=True,
            stdout=True,
            stderr=True,
            tty=True,
            workdir="/mnt/data"
        )['Id']

    def __del__(self):
        if self.exec_id:
            self.client.exec_remove(self.exec_id)

import uuid
def test_docker_wrapper():
    # Initialize a DockerProcWrapper instance with bash in interactive mode
    wrapper = DockerProcWrapper("python3 -i -q -u", session_path="/workspaces/open-interpreter/interpreter/dockerfiles/ses-" + str(uuid.uuid4()))

    print("Interactive Docker shell. Type 'exit' to quit.")

    while True:
        # Read user input
        command = input(">>> ")
        
        # Check for exit command
        if command.lower() == 'exit':
            break

        # Send the command to the Docker container
        wrapper.stdin.write(command + "\n")
        wrapper.stdin.flush()

        # Read from stdout and stderr
        print("Reading from stdout:")
        for line in iter(wrapper.stdout.readline, ""):
            print( line)

        print("Reading from stderr:")
        for line in iter(wrapper.stdout.readline, ""):
            print(f"STDERR: {line}")

    # Clean up
    del wrapper

if __name__ == '__main__':
    test_docker_wrapper()


Interactive Docker shell. Type 'exit' to quit.

Reading from stdout:

01000000000000271b5b3f32303034686e6f626f6479403635663464646136656261373a2f6d6e742f646174612420010000000000004365636
86f2073757070700d0a1b5b3f323030346c0d73757070700d0a1b5b3f32303034686e6f626f6479403635663464646136656261373a2f6d6e74
2f646174612420

nobody@65f4dda6eba7:/mnt/data$ echo suppp

suppp

nobody@65f4dda6eba7:/mnt/data$

KeyboardInterrupt: 

In [31]:
import gc
gc.collect()

0

In [18]:
import struct
from docker import DockerClient
import docker
import os
from rich import print
import re
import time
import queue
import select



class DockerStreamWrapper:
    def __init__(self, exec_id, stream_type, sock):
        self.exec_id = exec_id
        self.stream_type = stream_type
        self._sock = sock
        self._buffer = ""
        self._line_queue = queue.Queue()

    def demux_docker_stream(self, data):
        stdout = ""
        stderr = ""
        offset = 0
        while offset + 8 <= len(data):
            header = data[offset:offset + 8]
            stream_type, length = struct.unpack('>BxxxL', header)
            offset += 8
            chunk = data[offset:offset + length].decode('utf-8')
            offset += length
            if stream_type == 1:
                stdout += chunk
            elif stream_type == 2:
                stderr += chunk

        if stdout and not stdout.endswith('\n'):
            stdout += '\n'
        if stderr and not stderr.endswith('\n'):
            stderr += '\n'
        return stdout, stderr

    def readline(self, timeout=3):
        while self._line_queue.empty():
            ready_to_read, _, _ = select.select([self._sock], [], [], timeout)
            
            if not ready_to_read:
                return ""

            raw_data = self._sock.recv(2048)
            stdout, stderr = self.demux_docker_stream(raw_data)
            new_data = stdout if self.stream_type == 'stdout' else stderr
            self._buffer += new_data

            while '\n' in self._buffer:
                newline_pos = self._buffer.find('\n')
                line = self._buffer[:newline_pos]
                self._buffer = self._buffer[newline_pos + 1:]
                self._line_queue.put(line.strip())

        return self._line_queue.get()


    def write(self, data):
        commands = []
        current_command = ""
        escape_next = False  # Flag to indicate if the next character is part of an escape sequence

        for char in data:
            if escape_next:
                if char == 'n':  # Newline
                    if current_command.strip():
                        commands.append(current_command.strip())
                    current_command = ""
                elif char == 'r':  # Carriage return
                    current_command = ""
                elif char == '\\':  # Backslash
                    current_command += '\\\\'
                else:  # Any other escaped character (e.g., \t, \', \")
                    current_command += '\\' + char
                escape_next = False
            else:
                if char == '\\':
                    escape_next = True
                else:
                    current_command += char

        # Add the last command if it's not empty and if it's not just a newline character.
        if current_command.strip():
            commands.append(current_command.strip())
        
        for command in commands:
            self._sock.sendall(command.encode())
            time.sleep(1)

    def flush(self):
        pass


class DockerProcWrapper:
    def __init__(self, command, session_path):
        self.client = docker.APIClient()
        self.image_name = "openinterpreter-runtime-container:latest"
        self.session_path = session_path
        self.id = os.path.basename(session_path)
        self.lang = self.extract_language_from_command(command)
        self.exec_id = None
        self.exec_socket = None

        # Initialize container
        self.init_container()

        self.init_exec_instance(command)
        

        self.stdin = DockerStreamWrapper(
            self.exec_id, 'stdin', self.exec_socket)
        self.stdout = DockerStreamWrapper(
            self.exec_id, 'stdout', self.exec_socket)
        self.stderr = DockerStreamWrapper(
            self.exec_id, 'stderr', self.exec_socket)

    def init_container(self):
        self.container = None
        try:
            containers = self.client.containers(
                filters={"label": f"session_id={self.id}"}, all=True)
            if containers:
                self.container = containers[0]
                container_id = self.container.get('Id')
                container_info = self.client.inspect_container(container_id)
                if container_info.get('State', {}).get('Running') is False:
                    print(container_info.get('State', {}))
                    self.client.start(container=container_id)
                    print("waiting for container start")
                    self.wait_for_container_start(container_id)
                    print("container started")
            else:
                volume_binding = {
                    self.session_path: {'bind': '/mnt/data', 'mode': 'rw'}
                }
                env_client = docker.from_env()
                self.container = env_client.containers.run(
                self.image_name,
                detach=True,
                command="/bin/bash -i",  # Keep the container running with bash
                labels={'session_id': self.id},
                volumes={self.session_path: {'bind': '/mnt/data', 'mode': 'rw'}},
                user="nobody",  # Run as non-root user for security,
                stdin_open = True,
                tty=True
            )
                self.client.start(container=self.container.id)
                print("waiting for container start")
                self.wait_for_container_start(self.container.id)
                print("container started")

        except Exception as e:
            print(f"An error occurred: {e}")

    def init_exec_instance(self, command):
        if self.container:
            self.exec_id = self.client.exec_create(
                self.container.id,
                cmd=command,
                stdin=True,
                stdout=True,
                stderr=True,
                workdir="/mnt/data",
                user="nobody",
                tty=True

            )['Id']
            self.exec_socket = self.client.exec_start(
                self.exec_id, socket=True, tty=False, demux=True)._sock
            
            self._exec_socket 
            print(f"Attributes and Methods of {self.exec_socket.__class__.__name__}:")
            for attr_name in dir(self.exec_socket):
                # Filter out special methods and attributes (those starting with "__")
                if not attr_name.startswith("__"):
                    attr_value = getattr(self.exec_socket, attr_name)
                    docstring = attr_value.__doc__
                    print(f"{attr_name}: \n {docstring}\n")
            

    @staticmethod
    def extract_language_from_command(command):
        # Normalize the command to lower case for easier searching
        command_lower = command.lower()

        # Extract Python
        if "python" in command_lower or os.path.basename(command_lower).startswith("python"):
            return "python"

        # Extract R
        if re.search(r'\bR\b', command):
            return "r"

        # Extract Shell
        if any(shell in command_lower for shell in ["bash", "sh", "zsh", "fish"]):
            return "shell"

        # Extract Node.js
        if "node" in command_lower:
            return "javascript"

        # Return unknown if we can't determine the language
        return "unknown"

    def wait_for_container_start(self, container_id, timeout=30):
        start_time = time.time()
        while True:
            container_info = self.client.inspect_container(container_id)
            if container_info.get('State', {}).get('Running') is True:
                return True
            elif time.time() - start_time > timeout:
                raise TimeoutError(
                    "Container did not start within the specified timeout.")
            time.sleep(1)

import threading
import uuid
import uuid
def test_docker_wrapper():
    # Initialize a DockerProcWrapper instance with bash in interactive mode
    wrapper = DockerProcWrapper("python3 ", session_path="/workspaces/open-interpreter/interpreter/dockerfiles/ses-" + str(uuid.uuid4()))

    print("Interactive Docker shell. Type 'exit' to quit.")

    while True:
        # Read user input
        command = input(">>> ")
        
        # Check for exit command
        if command.lower() == 'exit':
            break

        # Send the command to the Docker container
        wrapper.stdin.write(command + "\n")
        wrapper.stdin.flush()

        # Read from stdout and stderr until an empty line is received
        print("Reading from stdout:")
        while True:
            line = wrapper.stdout.readline()
            if line == ">>>":
                break
            print(f"STDOUT: {line}")

        print("Reading from stderr:")
        while True:
            line2 = wrapper.stderr.readline()
            if not line2:
                break
            print(f"STDERR: {line2}")

    # Clean up
    del wrapper

if __name__ =='__main__':
    test_docker_wrapper()


waiting for container start

container started

AttributeError: 'DockerProcWrapper' object has no attribute '_exec_socket'

In [15]:
class DockerStreamWrapper:
    def __init__(self, exec_id, stream_type, sock):
        self.exec_id = exec_id
        self.stream_type = stream_type
        self._sock = sock
        self._buffer = ""
        self._line_queue = queue.Queue()

    @staticmethod
    def demux_docker_stream(data):
        stdout = ""
        stderr = ""
        offset = 0
        while offset + 8 <= len(data):
            header = data[offset : offset + 8]
            (stream_type, length) = struct.unpack(">BxxxL", header)
            offset += 8
            chunk = data[offset : offset + length].decode("utf-8")
            offset += length
            if stream_type == 1:
                stdout += chunk
            elif stream_type == 2:
                stderr += chunk
        if stdout and (not stdout.endswith("\n")):
            stdout += "\n"
        if stderr and (not stderr.endswith("\n")):
            stderr += "\n"
        return (stdout, stderr)

    def readline(self, timeout=3):
        while self._line_queue.empty():
            ready_to_read, _, _ = select.select([self._sock], [], [], timeout)
            
            if not ready_to_read:
                return ""

            raw_data = self._sock.recv(2048)
            print(raw_data)
            stdout, stderr = self.demux_docker_stream(raw_data)
            print((stdout, stderr))
            new_data = stdout if self.stream_type == 'stdout' else stderr
            self._buffer += new_data

            while '\n' in self._buffer:
                newline_pos = self._buffer.find('\n')
                line = self._buffer[:newline_pos]
                self._buffer = self._buffer[newline_pos + 1:]
                self._line_queue.put(line.strip())

        return self._line_queue.get()


    def write(self, data):
        self._sock.sendall(data.encode())

    @staticmethod
    def flush():
        pass


class DockerProcWrapper:
    def __init__(self, command, session_path):
        self.client = docker.APIClient()
        self.image_name = "openinterpreter-runtime-container:latest"
        self.session_path = session_path
        self.id = os.path.basename(session_path)
        self.lang = self.extract_language_from_command(command)
        self.exec_id = None
        self.exec_socket = None

        # Initialize container
        self.init_container()

        self.init_exec_instance(command)

        self.stdin = DockerStreamWrapper(self.exec_id, "stdin", self.exec_socket)
        self.stdout = DockerStreamWrapper(self.exec_id, "stdout", self.exec_socket)
        self.stderr = DockerStreamWrapper(self.exec_id, "stderr", self.exec_socket)

    def init_container(self):
        self.container = None
        try:
            containers = self.client.containers(
                filters={"label": f"session_id={self.id}"}, all=True
            )
            if containers:
                self.container = containers[0]
                container_id = self.container.get("Id")
                container_info = self.client.inspect_container(container_id)
                if container_info.get("State", {}).get("Running") is False:
                    
                    self.client.start(container=container_id)
                    print("waiting for container start")
                    self.wait_for_container_start(container_id)
                    
            else:
                env_client = docker.from_env()
                self.container = env_client.containers.run(
                    self.image_name,
                    detach=True,
                    command="/bin/bash -i",  # Keep the container running with bash
                    labels={"session_id": self.id},
                    volumes={self.session_path: {"bind": "/mnt/data", "mode": "rw"}},
                    user="nobody",  # Run as non-root user for security,
                    stdin_open=True,
                    tty=True,
                )
                self.client.start(container=self.container.id)
                print("waiting for container start")
                self.wait_for_container_start(self.container.id)
                print("container started")

        except Exception as e:
            print(f"An error occurred: {e}")

    def init_exec_instance(self, command):
        if self.container:
            self.exec_id = self.client.exec_create(
                self.container.id,
                cmd=command,
                stdin=True,
                stdout=True,
                stderr=True,
                workdir="/mnt/data",
                user="nobody",
                tty=True,
            )["Id"]
            self.exec_socket = self.client.exec_start(
                self.exec_id, socket=True, tty=True, demux=True
            )._sock

    @staticmethod
    def extract_language_from_command(command):
        # Normalize the command to lower case for easier searching
        command_lower = command.lower()

        # Extract Python
        if "python" in command_lower or os.path.basename(command_lower).startswith("python"):
            return "python"

        # Extract R
        if re.search(r"\bR\b", command):
            return "r"

        # Extract Shell
        if any(shell in command_lower for shell in ["bash", "sh", "zsh", "fish"]):
            return "shell"

        # Extract Node.js
        if "node" in command_lower:
            return "javascript"

        # Return unknown if we can't determine the language
        return "unknown"

    def wait_for_container_start(self, container_id, timeout=30):
        start_time = time.time()
        while True:
            container_info = self.client.inspect_container(container_id)
            if container_info.get("State", {}).get("Running") is True:
                return True
            if time.time() - start_time > timeout:
                raise TimeoutError("Container did not start within the specified timeout.")
            time.sleep(1)



import threading
import uuid
import uuid
def test_docker_wrapper():
    # Initialize a DockerProcWrapper instance with bash in interactive mode
    wrapper = DockerProcWrapper("python3 -i -q  ", session_path="/workspaces/open-interpreter/interpreter/dockerfiles/ses-" + str(uuid.uuid4()))

    print("Interactive Docker shell. Type 'exit' to quit.")

    while True:
        # Read user input
        command = input(">>> ")
        
        # Check for exit command
        if command.lower() == 'exit':
            break

        # Send the command to the Docker container
        wrapper.stdin.write(command)
        wrapper.stdin.flush()

        # Read from stdout and stderr until an empty line is received
        print("Reading from stdout:")
        while True:
            line = wrapper.stdout.readline()
            if not line:
                break
            print(f"STDOUT: {line}")

        print("Reading from stderr:")
        while True:
            line2 = wrapper.stderr.readline()
            if not line2:
                break
            print(f"STDERR: {line2}")

    # Clean up
    del wrapper

if __name__ =='__main__':
    test_docker_wrapper()

waiting for container start

container started

Interactive Docker shell. Type 'exit' to quit.

Reading from stdout:

b'>>> print("hello")'

('', '')

Reading from stderr:

KeyboardInterrupt: Interrupted by user

In [31]:
import os
import docker

base_url = os.environ.get('DOCKER_HOST')
print(base_url)
api_version = os.environ.get('DOCKER_API_VERSION')
print(api_version)
tls_config = None

# If DOCKER_TLS_VERIFY is set, setup TLS configuration
if os.environ.get('DOCKER_TLS_VERIFY') == '1':
    print("using tls")
    cert_path = os.environ.get('DOCKER_CERT_PATH', '.')
    tls_config = docker.tls.TLSConfig(
        client_cert=(os.path.join(cert_path, 'cert.pem'), os.path.join(cert_path, 'key.pem')),
        verify=os.path.join(cert_path, 'ca.pem')
    )

api_client = docker.APIClient(base_url=base_url, version=api_version, tls=tls_config)

# Now you can use `api_client` just like you would use a client initialized with `from_env`.
api_client.base_url

None

None

'http+docker://localhost'

In [21]:
from socket import socket, AF_INET, SOCK_STREAM
from socketio import SocketIO

# Create a dummy socket
s = socket(AF_INET, SOCK_STREAM)
s.bind(('localhost', 0))  # Bind to a random available port
s.listen(1)

# Accept a connection (this will block until a connection is made)
conn, addr = s.accept()

# Wrap the socket into a SocketIO object
sock_io = SocketIO(conn.fileno(), 'rb+')

# List the methods and attributes
print(dir(sock_io))

# Close the socket
s.close()


ModuleNotFoundError: No module named 'socketio'

In [9]:
client.containers()

[{'Id': '003c797f6415d4bc910e316af6d65f6784981494d1245e51d2d5f3184649e00f',
  'Names': ['/exciting_cerf'],
  'Image': 'openinterpreter-runtime-container',
  'ImageID': 'sha256:dfbbd697df87dd2ea2ab5226a9c943dc377259412c8238f449ceaa09ebd6fedf',
  'Command': 'bash',
  'Created': 1694967014,
  'Ports': [],
  'Labels': {},
  'State': 'running',
  'Status': 'Up About a minute',
  'HostConfig': {'NetworkMode': 'default'},
  'NetworkSettings': {'Networks': {'bridge': {'IPAMConfig': None,
     'Links': None,
     'Aliases': None,
     'NetworkID': '8bfbe968b594c1386e69b6defc87071154f71b19d7541fb6219f3fa9e91042e1',
     'EndpointID': 'aa3575fa699a117796bd9443ba34a381680f39e2a9297ffe65ed681c86b16d13',
     'Gateway': '172.17.0.1',
     'IPAddress': '172.17.0.2',
     'IPPrefixLen': 16,
     'IPv6Gateway': '',
     'GlobalIPv6Address': '',
     'GlobalIPv6PrefixLen': 0,
     'MacAddress': '02:42:ac:11:00:02',
     'DriverOpts': None}}},
  'Mounts': []}]